<a href="https://colab.research.google.com/github/ichekhovskikh/recommendation-system/blob/master/RecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import gensim
import os
import collections
import smart_open
import random
import json

from nltk.corpus import stopwords
from pymystem3 import Mystem

#Начинаем
Для начала нам понадобится комплект документов для обучения нашей модели doc2vec. Теоретически, документ может быть чем угодно: коротким твитом из 140 символов, отдельным абзацем, новостной статьей или книгой. В NLP комплект документов часто называют корпусом.

Будем тренировать нашу модель на собственном корпусе. Этот корпус содержит 70 текстов.

И мы проверим нашу модель на глаз, используя тестовый корпус, который содержит 7 документов.

Dataset состоит из трех строк: id (идентификатор строки), text (текст статьи), tag (идентификаторы похожих статей)

In [0]:
# Set file names for train and test data
train_path = 'https://raw.githubusercontent.com/ichekhovskikh/recommendation-system/master/train.csv'
test_path = 'https://raw.githubusercontent.com/ichekhovskikh/recommendation-system/master/test.csv'

## Опредлим функцию для чтения и предварительной обработки текста
Ниже мы определяем функцию для открытия  train/test файла, предварительно обрабатываем каждый текст датасета, используя простой инструмент предварительной обработки gensim (то есть, разбиваем текст на отдельные слова, удалите знаки препинания, установите строчные буквы и т. д.) и возвращаем список слов. Для обучения модели нам нужно будет связать тег/номер с каждым документом учебного корпуса. В нашем случае тег - это похожия статья.

In [0]:
def lemmatize(text):
    return Mystem().lemmatize(text\)

In [0]:
def remove_stopwords(text):
    russian_stopwords = stopwords.words("russian")
    return [word for word in text if word not in russian_stopwords]

In [0]:
def read_corpus(corpus_path, tokens_only=False):
    with open(corpus_path) as corpus_file:  
        corpus = json.load(corpus_file)
        for article in corpus:
            normalized_text = gensim.utils.simple_preprocess(article['text'])
            normalized_text = lemmatize(normalized_text)
            normalized_text = remove_stopwords(normalized_text)
            if tokens_only:
                yield normalized_text
            else:
                yield gensim.models.doc2vec.TaggedDocument(normalized_text, article['tags'])

In [0]:
def get_article_text(index, corpus_path):
    with open(corpus_path) as corpus_file:  
        corpus = json.load(corpus_file)
        return corpus[index]['text']

In [0]:
train_corpus = list(read_corpus(train_path))
test_corpus = list(read_corpus(test_path, tokens_only=True))

Давайте посмотрим на учебный корпус:

In [0]:
print(train_corpus[:2])

[TaggedDocument(words=['относится', 'семье', 'языков', 'подобным', 'синтаксисом', 'из', 'них', 'его', 'синтаксис', 'наиболее', 'близок', 'java', 'язык', 'имеет', 'статическую', 'типизацию', 'поддерживает', 'полиморфизм', 'перегрузку', 'операторов', 'том', 'числе', 'операторов', 'явного', 'неявного', 'приведения', 'типа', 'делегаты', 'атрибуты', 'события', 'свойства', 'обобщённые', 'типы', 'методы', 'итераторы', 'анонимные', 'функции', 'поддержкой', 'замыканий', 'linq', 'исключения', 'комментарии', 'формате', 'xml', 'переняв', 'многое', 'от', 'своих', 'языков', 'pascal', 'модула', 'smalltalk', 'особенности', 'java', 'опираясь', 'на', 'практику', 'их', 'использования', 'исключает', 'некоторые', 'модели', 'себя', 'как', 'проблематичные', 'при', 'разработке', 'программных', 'систем', 'например', 'отличие', 'от', 'некоторых', 'других', 'языков', 'не', 'поддерживает', 'множественное', 'наследование', 'классов', 'между', 'тем', 'допускается', 'множественное', 'наследование', 'интерфейсов'], t

Корпус тестирования выглядит так:

In [0]:
print(test_corpus[:2])

[['язык', 'ассемблера', 'машинно', 'ориентированный', 'язык', 'низкого', 'уровня', 'его', 'команды', 'прямо', 'соответствуют', 'отдельным', 'командам', 'машины', 'или', 'их', 'также', 'он', 'может', 'предоставлять', 'дополнительные', 'возможности', 'облегчения', 'такие', 'как', 'макрокоманды', 'выражения', 'средства', 'обеспечения', 'модульности', 'программ', 'может', 'рассматриваться', 'как', 'автокод', 'расширенный', 'конструкциями', 'языков', 'высокого', 'уровня', 'является', 'существенно', 'платформо', 'зависимым', 'языки', 'ассемблера', 'для', 'различных', 'аппаратных', 'платформ', 'несовместимы', 'хотя', 'могут', 'быть', 'целом', 'подобны', 'русском', 'языке', 'может', 'именоваться', 'просто', 'ассемблером', 'типичны', 'выражения', 'типа', 'писать', 'программу', 'на', 'ассемблере', 'что', 'строго', 'говоря', 'неверно', 'так', 'как', 'ассемблером', 'именуется', 'утилита', 'трансляции', 'программы', 'языка', 'ассемблера', 'объектный', 'код', 'компьютера'], ['перитонит', 'развиваетс

Обратите внимание, что корпус тестирования представляет собой просто список списков и не содержит никаких тегов.

# Обучение модели
## Создание объекта Doc2Vec
Теперь мы создадим модель Doc2Vec с векторным размером 50 слов и перебираем учебный корпус 40 раз. Мы устанавливаем минимальную длину слова равной двум, чтобы отбрасывать слова с очень малым количеством вхождений.

Однако это очень маленький набор данных (303 документа) с короткими текстами (несколько сотен слов). Добавление пропусков может иногда помочь с такими маленькими наборами данных.

По сути, словарь - это словарь всех уникальных слов, извлеченных из учебного корпуса.

In [0]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(train_corpus)
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 455 ms, sys: 27.6 ms, total: 483 ms
Wall time: 486 ms


## Inferring a Vector
Важно отметить, что теперь вы можете вывести вектор для любого фрагмента текста без необходимости переобучать модель, передав список слов в функцию model.infer_vector. Затем этот вектор можно сравнить с другими векторами по косинусному подобию.

In [0]:
model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])

array([ 0.04902249, -0.16510253,  0.11363754, -0.6399486 , -0.15127012,
       -0.39009207,  0.23565254,  0.00360098,  0.27722365,  0.27949998,
        0.39558393,  0.42618063, -0.17123514,  0.36964968,  0.6549083 ,
        0.27549383,  0.12051801, -0.6698091 , -0.26201582,  0.4271218 ,
        0.30213237, -0.29241943, -0.7244459 ,  0.66987437, -0.1077728 ,
        0.56507695,  0.52881694,  0.76878136,  0.8282877 ,  0.12631255,
       -0.48634398, -0.76747817,  0.31649363,  0.24880627,  0.09970368,
        1.0368389 ,  0.29331225, -0.25738555, -0.04301599, -0.61783844,
        0.02658267,  0.8510329 ,  0.36612463, -0.5975576 , -0.34572932,
        0.18082733, -0.76896995,  0.08389634, -0.5392681 ,  0.6814112 ],
      dtype=float32)

Обратите внимание, что infer_vector() принимает не строку, а список строковых токенов, которые уже должны были быть разбиты на токены так же, как свойство words исходных объектов обучающего документа.

Также обратите внимание, что, поскольку лежащие в основе алгоритмы обучения представляют собой проблему итеративной аппроксимации, в которой используется внутренняя рандомизация, повторные выводы одного и того же текста будут возвращать слегка разные векторы.

# Оценочная модель
Чтобы оценить нашу новую модель, мы сначала выведем новые векторы для каждого документа тренировочного корпуса, сравним выведенные векторы с тренировочным корпусом.

In [0]:
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


По сути, более 95% предполагаемых документов оказываются наиболее похожими на себя, и примерно в 5% случаев они по ошибке наиболее похожи на другие документы. проверка выведенного вектора по обучающему вектору является своего рода «проверкой работоспособности» в отношении того, ведет ли модель себя с пользой и согласованностью, хотя и не является реальным значением «точности».

Это здорово и не совсем удивительно. Мы можем взглянуть на пример:

In [0]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('1)', 0), ('2', 1), ('3)', 2), ('4)', 3)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (69): «горностай небольшой зверек семейства куньих очень ценится за свой мех из меха горностая шились мантии для королей царей простым людям иметь шубу или даже полушубок из меха этого животного было не просто не по карману но не по статусу длина этого животного не более см его хвост имеет длину до см летом окраска меха горностая на спине коричнево бурая животик бело желтоватый зимой же зверек становится чисто белым черным пятнышком на кончике хвоста»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3):

1) (69, 0.9298030138015747): «горностай небольшой зверек семейства куньих очень ценится за свой мех из меха горностая шились мантии для королей царей простым людям иметь шубу или даже полушубок из меха этого животного было не просто не по карману но не по статусу длина этого животного не более см его хвост имеет длину до см летом окраска меха горностая на спине коричнево бурая животик бело желтоватый зимой же зверек становится чисто белым черным пятнышком 

Обратите внимание, что наиболее похожий документ (как правило, тот же текст) имеет показатель сходства, приближающийся к 1,0. Тем не менее, оценка сходства для документов второго ранга должна быть значительно ниже (при условии, что документы на самом деле разные), и рассуждение становится очевидным, когда мы исследуем сам текст.

# Тестирование модели
Используя тот же подход, что и выше, мы выведем вектор для случайно выбранного тестового документа и сравним документ с нашей моделью на глаз.

In [0]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus) - 1)
inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('1)', 0), ('2', 1), ('3)', 2), ('4)', 3)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Test Document (4): «начав подражания классикам американского рок ролла годов the beatles пришли собственному стилю звучанию the beatles оказали значительное влияние на рок музыку признаются специалистами одной из наиболее успешных групп xx века как творческом так коммерческом смысле многие известные рок музыканты признают что стали таковыми под влиянием песен the beatles момента выпуска сингла please please me ask me why году группа начала восхождение успеху породив своим творчеством глобальное явление битломанию четвёрка стала первой британской группой пластинки которой завоевали популярность первые места чартах сша неё началось всемирное признание британских коллективов также ливерпульского merseybeat звучания рок музыки музыкантам группы их продюсеру звукорежиссёру джорджу мартину принадлежат новаторские разработки области звукозаписи комбинирования различных стилей включая симфоническую психоделическую музыку также съёмок видеоклипов»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
